<h2><span style="color:gray">ipyrad-analysis toolkit:</span> PCA</h2>

Principal components analysis is a dimensionality reduction method used to transform data points into fewer orthogonal axes that can explain the greatest amount of variance in the data.

PCA analyses are *very sensitive* to missing data. The `ipyrad.pca` tool makes it easy to perform PCA on RAD-seq data by filtering and/or imputing missing data, and allowing for easy subsampling of individuals to include in analyses. 

### Required software

In [1]:
# conda install ipyrad -c bioconda
# conda install scikit-learn -c bioconda
# conda install toyplot -c eaton-lab

In [2]:
import ipyrad.analysis as ipa
import toyplot

### Short Tutorial:


### Imputation algorithms:

We offer three algorithms for *imputing* missing data:

1. **sample**: Randomly sample genotypes based on the frequency of alleles within (user-defined) populations (imap).   


2. **kmeans**: Randomly sample genotypes based on the frequency of alleles in (kmeans cluster-generated) populations. 


3. **None**: All missing values are imputed with zeros (ancestral allele).

#### Input data file and population assignments
If you are using the "sample" input method then population assignments (imap dictionary) are used for for filtering, color coding plots, and for imputation. If you are using the "kmeans" imputing method then population assignments are only used for filtering and color coding plots.

In [3]:
# the path to your .snps.hdf5 database file
data = "/home/deren/Downloads/ref_pop2.snps.hdf5"

In [4]:
# group individuals into populations
imap = {
    "virg": ["TXWV2", "LALC2", "SCCU3", "FLSF33", "FLBA140"],
    "mini": ["FLSF47", "FLMO62", "FLSA185", "FLCK216"],
    "gemi": ["FLCK18", "FLSF54", "FLWO6", "FLAB109"],
    "bran": ["BJSL25", "BJSB3", "BJVL19"],
    "fusi": ["MXED8", "MXGT4", "TXGR3", "TXMD3"],
    "sagr": ["CUVN10", "CUCA4", "CUSV6", "CUMM5"],
    "oleo": ["CRL0030", "HNDA09", "BZBB1", "MXSA3017", "CRL0001"],
}

# require that 50% of samples have data in each group
minmap = {i: 0.5 for i in imap}

#### Enter data file and params
The `pca` analysis object takes input data as the *.snps.hdf5* file produced by ipyrad. All other parameters are optional. The **imap** dictionary groups individuals into populations and **minmap** can be used to filter SNPs to only include those that have data for at least some proportion of samples in every group. The **mincov** option works similarly, it filters SNPs that are shared across less than some proportion of all samples (in contrast to minmap this does not use imap groupings). 

When you init the object it will load the data and apply filtering. The printed output tells you how many SNPs were removed by each filter and the remaining amount of missing data after filtering. These remaining missing values are the ones that will be filled with imputation. 

In [5]:
# init pca object with input data and (optional) parameter options
pca = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap,
    mincov=0.25,
    impute_method="sample",
)

Samples: 29
Sites before filtering: 349914
Filtered (indels): 0
Filtered (bi-allel): 13379
Filtered (mincov): 10391
Filtered (minmap): 111825
Filtered (combined): 120177
Sites after filtering: 229737
Sites containing missing values: 219551 (95.57%)
Missing values in SNP matrix: 814369 (12.22%)
Imputation: 'sampled'; (0, 1, 2) = 77.3%, 10.7%, 12.0%


#### Run PCA and plot results. 
When you call `.run()` a PCA model will be fit to the data and two results will be returned: (1) samples weightings on the component axes; (2) the proportion of variance explained by each axis. Since the next step is typically to plot these values you can use the function `.run_and_plot()` to return the results as a toytree plot. The first two arguments to this are the two axes to be plotted.

In [6]:
# plot PC axes 0 and 2 with no subsampling
pca.run_and_plot_2D(0, 2, subsample=False);

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t073abae2173940ec9fbab121a64e5ba5" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> BJSB3 BJSL25 BJVL19 BZBB1 CRL0001 CRL0030 CUCA4 CUMM5 CUSV6 CUVN10 FLAB109 FLBA140 FLCK18 FLCK216 FLMO62 FLSA185 FLSF33 FLSF47 FLSF54 FLWO6 HNDA09 LALC2 MXED8 MXGT4 MXSA3017 SCCU3 TXGR3 TXMD3 TXWV2 -100 0 100 200 PC0 (14.0%) explained -100 0 100 PC2 (6.4%) explained virg mini gemi bran fusi sagr oleo

#### Subsampling SNPs
By default `run()` will randomly subsample one SNP per RAD locus to reduce the effect of linkage on your results. This can be turned off by setting `subsample=False`, like in the plot above. If using subsampling you can set the random seed to make your results repeatable. The results here subsampling from 226K SNPs to 29K, but the final results are quite similar.

In [7]:
# plot PC axes 0 and 2 with no subsampling
pca.run_and_plot_2D(0, 2, seed=123, subsample=True);

Subsampling SNPs: 29775/229737


<svg class="toyplot-canvas-Canvas" height="300.0px" id="t87f7ce92000745469772a28bb1e69e25" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> BJSB3 BJSL25 BJVL19 BZBB1 CRL0001 CRL0030 CUCA4 CUMM5 CUSV6 CUVN10 FLAB109 FLBA140 FLCK18 FLCK216 FLMO62 FLSA185 FLSF33 FLSF47 FLSF54 FLWO6 HNDA09 LALC2 MXED8 MXGT4 MXSA3017 SCCU3 TXGR3 TXMD3 TXWV2 -30 0 30 60 PC0 (14.8%) explained -25 0 25 50 PC2 (6.5%) explained virg mini gemi bran fusi sagr oleo

## Cookbook

<h2><span style="color:red"> Advanced:</span> Other imputation options</h2>

#### No imputation
The None option will almost always be a *bad choice* when there is any reasonable amount of missing data. Missing values will all be filled as zeros (ancestral allele). I show it here for comparison to the imputed results. The two points near the top of the plot are samples with the most missing data that are erroneously grouped together. The rest of the samples also form much less clear clusters than in the other examples where we use imputation or stricter filtering options.

In [8]:
# init pca object with input data and (optional) parameter options
no_imputation = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap,
    mincov=0.25,
    impute_method=None,
)
no_imputation.run_and_plot_2D(0, 2, seed=123, subsample=False);

Samples: 29
Sites before filtering: 349914
Filtered (indels): 0
Filtered (bi-allel): 13379
Filtered (mincov): 10391
Filtered (minmap): 111825
Filtered (combined): 120177
Sites after filtering: 229737
Sites containing missing values: 219551 (95.57%)
Missing values in SNP matrix: 814369 (12.22%)
Imputation (null; sets to 0): 100.0%, 0.0%, 0.0%


<svg class="toyplot-canvas-Canvas" height="300.0px" id="t9fadc8fe0d144dbc9e525c95b9fd0f28" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> BJSB3 BJSL25 BJVL19 BZBB1 CRL0001 CRL0030 CUCA4 CUMM5 CUSV6 CUVN10 FLAB109 FLBA140 FLCK18 FLCK216 FLMO62 FLSA185 FLSF33 FLSF47 FLSF54 FLWO6 HNDA09 LALC2 MXED8 MXGT4 MXSA3017 SCCU3 TXGR3 TXMD3 TXWV2 -50 0 50 100 150 PC0 (10.9%) explained -50 0 50 100 150 200 PC2 (5.6%) explained virg mini gemi bran fusi sagr oleo

#### No imputation but stricter filtering
Here I do not allow for any missing data (`mincov`=1.0). You can see that this reduces the number of total SNPs from 349K to 10K. The final reslult is not too different from our first example, but seems a little less smooth. In most data sets it is probably better to include more data by imputing some values, though.

In [9]:
# init pca object with input data and (optional) parameter options
strict_filtering = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap,
    mincov=1.0,
    impute_method=None,
)
strict_filtering.run_and_plot_2D(0, 2, seed=123, subsample=False);

Samples: 29
Sites before filtering: 349914
Filtered (indels): 0
Filtered (bi-allel): 13379
Filtered (mincov): 339412
Filtered (minmap): 111825
Filtered (combined): 339728
Sites after filtering: 10186
Sites containing missing values: 0 (0.00%)
Missing values in SNP matrix: 0 (0.00%)


<svg class="toyplot-canvas-Canvas" height="300.0px" id="t5447357e14e74a958883d71afb4d4d45" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> BJSB3 BJSL25 BJVL19 BZBB1 CRL0001 CRL0030 CUCA4 CUMM5 CUSV6 CUVN10 FLAB109 FLBA140 FLCK18 FLCK216 FLMO62 FLSA185 FLSF33 FLSF47 FLSF54 FLWO6 HNDA09 LALC2 MXED8 MXGT4 MXSA3017 SCCU3 TXGR3 TXMD3 TXWV2 -20 -10 0 10 20 30 PC0 (13.6%) explained -10 0 10 20 30 PC2 (5.5%) explained virg mini gemi bran fusi sagr oleo

### Kmeans imputation

The *kmeans* clustering method allows imputing values based on population allele frequencies (like the *sample* method) but without having to *a priori* assign individuals to populations. In other words, it is meant to reduce the bias introduced by assigning individuals yourself. Instead, this method uses kmeans clustering to group individuals into "populations" and then imputes values based on those population assignments. This is accomplished through **iterative clustering**, starting by using only SNPs that are present across 90% of all samples (this can be changed with the topcov param) and then allowing more missing data in each iteration until it reaches the mincov parameter value. 

This method works great especially if you have a lot of missing data and fear that user-defined population assignments will bias your results. Here it gives super similar results to our first plots using the "sample" impute method, suggesting that our population assignments are not greatly biasing the results. 

In [10]:
# kmeans imputation 
kmeans_imputation = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap,
    mincov=0.5,
    impute_method=7,
)
kmeans_imputation.run_and_plot_2D(0, 2, seed=123);

Kmeans clustering: iter=0, K=7, mincov=0.9, minmap={'global': 0.5}
Samples: 29
Sites before filtering: 349914
Filtered (indels): 0
Filtered (bi-allel): 13379
Filtered (mincov): 256313
Filtered (minmap): 30459
Filtered (combined): 259135
Sites after filtering: 90779
Sites containing missing values: 80593 (88.78%)
Missing values in SNP matrix: 127348 (4.84%)
Imputation: 'sampled'; (0, 1, 2) = 77.8%, 14.1%, 8.1%
{0: ['FLBA140', 'FLSF33', 'LALC2', 'SCCU3', 'TXWV2'], 1: ['BZBB1', 'CRL0001', 'CRL0030', 'HNDA09', 'MXSA3017'], 2: ['BJSB3', 'BJSL25', 'BJVL19'], 3: ['CUMM5'], 4: ['FLAB109', 'FLCK18', 'FLCK216', 'FLMO62', 'FLSA185', 'FLSF47', 'FLSF54', 'FLWO6'], 5: ['MXED8', 'MXGT4', 'TXGR3', 'TXMD3'], 6: ['CUCA4', 'CUSV6', 'CUVN10']}

Kmeans clustering: iter=1, K=7, mincov=0.8, minmap={0: 0.5, 1: 0.5, 2: 0.5, 3: 0.5, 4: 0.5, 5: 0.5, 6: 0.5}
Samples: 29
Sites before filtering: 349914
Filtered (indels): 0
Filtered (bi-allel): 13379
Filtered (mincov): 151647
Filtered (minmap): 278652
Filtered (comb

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t9e039f50f23143e88b125c0d640ace5d" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> BJSB3 BJSL25 BJVL19 BZBB1 CRL0001 CRL0030 CUCA4 CUMM5 CUSV6 CUVN10 FLAB109 FLBA140 FLCK18 FLCK216 FLMO62 FLSA185 FLSF33 FLSF47 FLSF54 FLWO6 HNDA09 LALC2 MXED8 MXGT4 MXSA3017 SCCU3 TXGR3 TXMD3 TXWV2 -30 0 30 60 PC0 (14.9%) explained -25 0 25 50 PC2 (6.5%) explained virg mini gemi bran fusi sagr oleo

### <h2><span style="color:red"> Advanced:</span> Save plot to PDF</h2>
You can save the figure as a PDF or SVG using a toyplot render function like below. 

In [11]:
import toyplot.pdf

# save returned values of the plot command
canvas, axes, mark = kmeans_imputation.run_and_plot_2D(0, 2, seed=123)

# pass the canvas object to render function
toyplot.pdf.render(canvas, "PCA-kmeans-7.pdf")

Subsampling SNPs: 29616/228101


### <h2><span style="color:red"> Advanced:</span> Missing data per sample
You can view the proportion of missing data per sample by accessing the `.missing` data table from your `pca` analysis object. You can see that most samples in this data set had 10% missing data or less, but a few had 20-70% missing data. You can hover your cursor over the plot above to see the sample names. It seems pretty clear that samples with huge amounts of missing data do not stand out at outliers in these plots like they did in the no-imputation plot. Which is great!

In [12]:
# .missing is a pandas DataFrame
kmeans_imputation.missing.sort_values(by="missing")

,missing
BJSL25,0.03
BJVL19,0.03
FLBA140,0.03
FLSF54,0.04
CRL0001,0.04
CRL0030,0.04
LALC2,0.04
CUVN10,0.06
FLAB109,0.06
MXGT4,0.07
